# Base de données répertoriant les nouveaux textes législatifs (depuis 2006) en France

### Table des matières

* [Récupération des données de Légifrance via une API](#section1)
    * [xxx](#section11)
    * [xxx](#section12)
* [Nettoyage des données de Légifrance](#section2)
    * [xxx](#section21)
* [Sauvegarde des tableaux de données finalisées](#section3)

Ce notebook contient les différents codes qui nous ont permis d'accéder aux données disponibles grâce à l'API Piste de Légifrance.

Sur l'API de Légifrance les requêtes doivent être faites sur un fonds, soit un filtre correspondant à une catégorie spécifique de la base de données de Légifrance. Les différents fonds incluent notamment LODA , qui regroupe les lois, les ordonnances, les décrets et les arrêtés, mais il y a aussi CODE pour les documents relatifs aux différents codes et ALL qui permet de faire une requête sur tous les fonds. 

Au début de notre récupération des données nos requêtes ont été effectuées sur le fond LODA, ensuite CODE et enfin ALL, avant de revenir sur LODA. Cet ordre n'est dû qu'à l'absence de documentation claire, au faible nombre de projets présents sur internet utilisant l'API de Légifrance et d'erreurs que l'on n'arrive pas à expliquer et qui nous empêchent d'avoir accès à l'intégralité des fonds. 

En fait, lorsqu'on effectue une requête, l'API ne nous renvoie qu'un nombre limité de résultats (100 maximum), donc il faut faire une boucle pour récupérer tous les documents. Les requêtes supérieures au 10001e éléments renvoient une erreur 503, soit une erreur du serveur. Malgré des recherches et des mails envoyés au support et à des personnes travaillant à l'AIFE (Agence pour l'Informatique Financière de l'État) sur l'API de Légifrance nous n'avons pas trouvé de solution à cela. 

Ainsi, pour l'avancement du projet, nous avons pris la décision de ne récupérer que les données sur le fond LODA, car ce fond est d'après nous le plus pertinent - les modifications des codes étant plus difficiles à étudier sous l'angle quantitatif par rapport aux lois, car il s'agit en partie de modification de vocabulaire employé.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import s3fs

In [2]:
# Chemin vers le fichier CSV
chemin_fichier = "data/resultats_legifrance_loda.csv"
df_loda = pd.read_csv(chemin_fichier)

In [3]:
df_loda.sample()

,Titre,ID,Date,Nature,Etat,Origine,Date Publication
3065,Arrêté du 22 mars 2016 portant approbation du ...,LEGITEXT000038810959_26-03-2016,2016-03-26T00:00:00.000+0000,ARRETE,VIGUEUR,LEGI,2016-03-25T00:00:00.000+0000


In [4]:
# On met la date sous format date de pandas
df_loda['Date'] = pd.to_datetime(df_loda['Date']) 

# On ajoute la colonne mois et l'année via la variable Date (On a arbitrairement écarté Date Publication car les écarts sont de quelques jours uniquement)
df_loda['Mois'] = df_loda['Date'].dt.month.apply(lambda x: f'{x:02}')
df_loda['Année'] = df_loda['Date'].dt.year.astype(str)

df_loda.head()

,Titre,ID,Date,Nature,Etat,Origine,Date Publication,Mois,Année
0,Arrêté du 31 août 2022 fixant les niveaux de p...,LEGITEXT000046244600_21-08-2023,2022-09-01 00:00:00+00:00,ARRETE,VIGUEUR,LEGI,2022-09-01T00:00:00.000+0000,09,2022
1,Arrêté du 30 août 2022 portant création de la ...,LEGITEXT000046664861_28-08-2023,2022-12-02 00:00:00+00:00,ARRETE,VIGUEUR,LEGI,2022-12-01T00:00:00.000+0000,12,2022
2,Arrêté du 30 août 2022 portant création de la ...,LEGITEXT000046664861_02-12-2022,2022-12-02 00:00:00+00:00,ARRETE,VIGUEUR,LEGI,2022-12-01T00:00:00.000+0000,12,2022
3,Arrêté du 29 août 2022 portant création de la ...,LEGITEXT000046666903_02-12-2022,2022-12-02 00:00:00+00:00,ARRETE,VIGUEUR,LEGI,2022-12-01T00:00:00.000+0000,12,2022
4,Arrêté du 22 août 2022 fixant la liste des pos...,LEGITEXT000046230032_21-11-2022,2022-09-01 00:00:00+00:00,ARRETE,VIGUEUR,LEGI,2022-08-31T00:00:00.000+0000,09,2022


In [5]:
# Vérification pertinence des données
print("Valeurs uniques dans la colonne 'année' :")
print(df_loda['Année'].unique())

print("\nValeurs uniques dans la colonne 'mois' :")
print(df_loda['Mois'].unique())

print("\nValeurs uniques et leur fréquence dans la colonne 'Nature' :")
df_loda['Nature'].value_counts()


Valeurs uniques dans la colonne 'année' :
['2022' '2023' '2024' '2021' '2020' '2019' '2222' '2018' '2017' '2016'
 '2015' '2014' '2013' '2012' '2011' '2010' '2009' '2008' '2007' '2006']

Valeurs uniques dans la colonne 'mois' :
['09' '12' '08' '10' '07' '01' '05' '06' '04' '03' '02' '11']

Valeurs uniques et leur fréquence dans la colonne 'Nature' :


Nature
ARRETE        6790
DECRET        1826
LOI           1207
ORDONNANCE     174
DECISION         3
Name: count, dtype: int64

On se permet d'exclure pour le reste de l'étude les nouvelles normes législatives adoptées de type "décision" puisqu'elles ne sont qu'au nombre de 3 sur la période étudiée (trop faible et donc sûrement pas significatif pour le reste de l'analyse).

In [6]:
df_loda = df_loda[df_loda['Nature'] != 'DECISION']

### Sauvegarde des tableaux de données finalisées <a class="anchor" id="section3"></a>

In [ ]:
'''
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

MY_BUCKET = "williamolivier"
fs.ls(MY_BUCKET)

# Sauvegarde dans S3
MY_BUCKET = "williamolivier"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/df_loda.csv"

with fs.open(FILE_PATH_OUT_S3, "w") as file_out:
    df_loda.to_csv(file_out)

fs.ls(f"{MY_BUCKET}/diffusion")
'''

PermissionError: Access Denied.